In [1]:
from torchvision import transforms
from PIL import Image

import time
import numpy as np

import torch
import torchvision

from tvm.contrib.download import download_testdata

model_name = "resnet18"
model = torchvision.models.__dict__[model_name](pretrained=True)
model = model.eval()

input = torch.randn(1, 3, 224, 224)
script_module = torch.jit.trace(model, input).eval()


image_path = download_testdata("https://github.com/dmlc/mxnet.js/blob/main/data/cat.png?raw=true",
                               "cat.png", module="data")
img = Image.open(image_path).resize((224, 224))

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

data = transform(img)

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# print(img)
img = np.expand_dims(img, axis=0)
print(img.shape)
img = np.transpose(img, (0, 3, 1, 2))

print(img.shape)

(1, 224, 224, 3)
(1, 3, 224, 224)


In [3]:
from tvm import relay

input_name = "input0"
shape_list = [(input_name, img.shape)]

mod,param = relay.frontend.from_pytorch(script_module, shape_list)

In [ ]:
shape_list

In [ ]:
mod

In [ ]:
param

In [4]:
import tvm 

target = 'llvm'
target_host = 'llvm'
ctx = tvm.cpu(0)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, target_host=target_host,params=param)

/opt/homebrew/lib/python3.11/site-packages/tvm-0.16.dev1105+g05e218c54-py3.11-macosx-13-arm64.egg/tvm/target/target.py:375: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [ ]:
lib.__dict__

In [ ]:
lib['default']

In [5]:
# from tvm.contrib import graph_runtime
import tvm.contrib.graph_executor as runtime

def bench_tvm():
    for i in range(100):
        dtype = "float32"

        # m = graph_runtime.GraphModule(lib['default'](ctx))
        m = runtime.GraphModule(lib['default'](ctx))
        m.set_input(input_name, tvm.nd.array(img.astype(dtype)))
        m.run()

        _ = m.get_output(0)

%timeit bench_tvm()

3.52 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
type(lib)

In [ ]:
lib.export_library("module.so")

In [ ]:
def bench_torch():
    for i in range(100):
        torch_img = torch.from_numpy(img.astype(np.float32))
        _ = model(torch_img)

%timeit bench_torch()

In [6]:
import tvm 

target = 'llvm'
target_host = 'llvm'
ctx = tvm.cpu(0)

with tvm.transform.PassContext(opt_level=3):
    graph, lib, params  = relay.build_module.build(mod, target=target, target_host=target_host,params=param)

/var/folders/y5/ssh5ylsd4tv7wfmm_06fmrqw0000gp/T/ipykernel_19500/2322423639.py:8: DeprecationWarning: legacy graph executor behavior of producing json / lib / params will be removed in the next release. Please see documents of tvm.contrib.graph_executor.GraphModule for the  new recommended usage.
  graph, lib, params  = relay.build_module.build(mod, target=target, target_host=target_host,params=param)


In [ ]:
lib.export_library('resnet18.so')

In [ ]:
with open("resnet18.graph","w") as f:
    f.write(graph)

In [ ]:
from tvm  import runtime

with open("resnet18.params","wb") as f:
    f.write(runtime.save_param_dict(params))